# 17A: Market Segmentation

Market segmentation can be used to determine whether the impact of other variables is different among population groups. The most common approach to market segmentation is for the analyst to consider sample segments which are mutually exclusive and collectively exhaustive (that is, each case is included in one and only one segment). Models are estimated for the sample associated with each segment and compared to the pooled model (all segments represented by a single model) to determine if there are statistically significant and important differences among the market segments.

Model 17A segments the market by automobile ownership for households that have one or fewer cars. It is appealing to include a distinct segment for households with no cars since the mode choice behavior of this segment is very different from the rest of the population due to their dependence on non-automobile modes. However, the size of this segment in the dataset is too small, so it is joined with the one car group. (pp. 129-133)5)

In [ ]:
# TEST
import larch

In [ ]:
import larch

larch.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = larch.examples.MTC(format="dataset")
d

Include only the cases where number of vehicles is 1 or less

In [ ]:
d = d.sel(caseid=d.numveh <= 1)

In [ ]:
m = larch.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import P, X

for a in [4, 5, 6]:
    m.utility_co[a] += X("hhinc") * P(f"hhinc#{a}")

Since the model we want to create groups together DA, SR2 and SR3+ jointly as reference alternatives with respect to income, we can simply omit all of these alternatives from the block that applies to hhinc.

For vehicles per worker, the preferred model include a joint parameter on SR2 and SR3+, but not including DA and not fixed at zero. Here we might use a shadow_parameter (also called an alias in some places), which allows us to specify one or more parameters that are simply a fixed proportion of another parameter. For example, we can say that vehbywrk_SR2 will be equal to vehbywrk_SR.

In [ ]:
for i in d["alt_names"][1:3]:
    name = str(i.values)
    a = int(i.altid)
    m.utility_co[a] += (
        +X("vehbywrk") * P("vehbywrk_SR")
        + X("wkccbd+wknccbd") * P("wkcbd_" + name)
        + X("wkempden") * P("wkempden_" + name)
        + P("ASC_" + name)
    )

for i in d["alt_names"][3:]:
    name = str(i.values)
    a = int(i.altid)
    m.utility_co[a] += (
        +X("vehbywrk") * P("vehbywrk_" + name)
        + X("wkccbd+wknccbd") * P("wkcbd_" + name)
        + X("wkempden") * P("wkempden_" + name)
        + P("ASC_" + name)
    )

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

In [ ]:
m.utility_ca = (
    +X("totcost/hhinc") * P("costbyincome")
    + X("tottime * (altid <= 4)") * P("motorized_time")
    + X("tottime * (altid >= 5)") * P("nonmotorized_time")
    + X("ovtt/dist * (altid <= 4)") * P("motorized_ovtbydist")
)

The “totcost/hhinc” data is computed once as a new variable when loading the model data. The same applies for tottime filtered by motorized modes (we harness the convenient fact that all the motorized modes have identifying numbers 4 or less), and “ovtt/dist”.

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 17A, Segmented for 1 or fewer cars"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 25, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(25)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True, options={"maxiter": 1000, "ftol": 1e-10})
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        ".*cost.*",
        ".*time.*",
        ".*dist.*",
    ),
    (
        "Zonal",
        "wkcbd.*",
        "wkempden.*",
    ),
    (
        "Household",
        "hhinc.*",
        "vehbywrk.*",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
revealed_x = dict(zip(m.pnames, result.x))

In [ ]:
# TEST
from pytest import approx

expected_x = {
    "ASC_Bike": 0.9742143708899291,
    "ASC_SR2": 0.5953670936985695,
    "ASC_SR3+": -0.7824827134037493,
    "ASC_Transit": 2.2585074587816134,
    "ASC_Walk": 2.9039591287055986,
    "costbyincome": -0.02264018916095553,
    "hhinc#4": -0.006444094209484317,
    "hhinc#5": -0.011725009048022181,
    "hhinc#6": -0.01198956540981421,
    "motorized_ovtbydist": -0.11308926104452616,
    "motorized_time": -0.021067655041764506,
    "nonmotorized_time": -0.04394234551517281,
    "vehbywrk_Bike": -2.644433522672152,
    "vehbywrk_SR": -3.0157086560458417,
    "vehbywrk_Transit": -3.9631693279363467,
    "vehbywrk_Walk": -3.3398299003269862,
    "wkcbd_Bike": 0.371887416799123,
    "wkcbd_SR2": 0.37071617423118847,
    "wkcbd_SR3+": 0.22893265840284804,
    "wkcbd_Transit": 1.1056371095671524,
    "wkcbd_Walk": 0.030612758978009455,
    "wkempden_Bike": 0.001542900697956932,
    "wkempden_SR2": 0.00204432338331191,
    "wkempden_SR3+": 0.0035300285386588638,
    "wkempden_Transit": 0.00316015805069698,
    "wkempden_Walk": 0.0037858816800672803,
}
for k in expected_x:
    assert revealed_x[k] == approx(expected_x[k], 2e-2), (
        f"{k}, {revealed_x[k] / expected_x[k]}"
    )